In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import get_peft_model, PeftModel, PeftConfig

In [26]:

model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b", device_map = "auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
config = PeftConfig.from_pretrained('/data/chris/adapters/peft-alpaca-lora-7b')
model = PeftModel.from_pretrained(model, '/data/chris/adapters/peft-alpaca-lora-7b', adapter_name="alpaca-lora")

In [28]:
model.load_adapter("/data/chris/adapters/peft-portuguese-llama", adapter_name="portuguese-llama")

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.original_module.weight', 'base_model.model.model.embed_tokens.modules_to_save.portuguese-llama.weight', 'base_model.model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.alpaca-lora.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.alpaca-lora.weight', 'base_model.model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.alpaca-lora.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.portuguese-llama.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.alpaca-lora.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.portuguese-llama.weight', 'base_model.model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.alpaca-lora.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.alpaca-lora.weight', 'base_model.mode

In [29]:
import torch

device = "cuda"


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=256,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [30]:
model.set_adapter("alpaca-lora")

In [31]:
instruction = "Tell me about alpacas."

print(evaluate(instruction))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:7! (when checking argument for argument index in method wrapper_CUDA__index_select)